In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
TRAIN_DATA_DIR = 'Food/train/'
VALIDATION_DATA_DIR = 'Food/val/'

NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [6]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 2284 images belonging to 2 classes.
Found 254 images belonging to 2 classes.


In [7]:
TRAIN_SAMPLES = 2284
VALIDATION_SAMPLES = 254

In [ ]:
def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:-6]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.3)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [ ]:
from collections import Counter

counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

In [ ]:
print(class_weights)

In [ ]:
model = model_maker()
model.compile(loss='categorical_crossentropy', #FocalLoss(alpha=1)
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model.fit_generator(
    train_generator,
    class_weight=class_weights,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=50,
    validation_data=validation_generator)

In [ ]:
test_data.to_csv("Predictions.csv",index=False)

# Saving Model

In [ ]:
import time
t = time.time()

#export_path = "saved_models/MobileNet/{}".format(int(t))
#model.save(export_path)
tf.saved_model.save(model,'HotDog')

export_path

# ios Model Conversion

In [ ]:
from tensorflow.keras.models import load_model
import coremltools

In [ ]:
model = load_model(export_path)

In [ ]:
model.summary()

In [ ]:
coreml_model = coremltools.convert(model)
coreml_model.save("HotDog.mlmodel")

# tflite

In [8]:
loaded = tf.saved_model.load('HotDog')

In [9]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'input_2': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_2')})
{'dense_1': TensorSpec(shape=(None, 2), dtype=tf.float32, name='dense_1')}


In [10]:
converter = tf.lite.TFLiteConverter.from_saved_model('HotDog')

In [11]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [13]:
def representative_data_gen():
    i = 0
    max_batches = math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE)
    for input_value, _ in validation_generator:
        if i < max_batches:
            yield [input_value]
            i= i+1
        else:
            break

In [14]:
converter.representative_dataset = representative_data_gen

In [15]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

In [ ]:
tflite_model = converter.convert()
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)